In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn  as sns
import pickle
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
d1=pd.read_csv('2018_Data.csv')
d2=pd.read_csv('2019_Data.csv')
df = pd.concat([d1,d2]).reset_index(drop = True)
df1=df.copy()
df1['time']=" "
df1.time = (pd.to_datetime(df['Hour'].astype(str) + ':' + df['Minute'].astype(str), format='%H:%M')
          .dt.time).astype(str)
df1['time']=df1['time'].str.extract(r'(\d{2}:\d{2})')
df1['time']=df1['time'].replace(['07:00','07:30','08:00','08:30','09:00','09:30','10:00','10:30','11:00','11:30'],1)
df1['time']=df1['time'].replace(['12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30','16:00','16:30','17:00','17:30'],2)
df1['time']=df1['time'].replace(['18:00','18:30','19:00','19:30'],3)
df1['time']=df1['time'].replace(['20:00','20:30','21:00','21:30','22:00','22:30','23:00','23:30','00:00','00:30','01:00','01:30','02:00','02:30','03:00','03:30','04:00','04:30','05:00','05:30','06:00','06:30'],0)
X=df1.drop(['Year','Month','Day','Hour','Minute','Clearsky GHI','GHI'],axis=1)
Y=df1['GHI']
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pci=pca.fit_transform(X)
itemspca = pd.DataFrame(pci)
itemspca.columns = ['PC{}'.format(i + 1) for i in range(pci.shape[1])]
itemspca.index = df.index
pca_xtrain = itemspca[:27000]
pca_xtest = itemspca[27000:]
from sklearn.ensemble import RandomForestRegressor
X = pca_xtrain.copy()
y = df['GHI']
X_test = pca_xtest.copy()
X_train = pca_xtrain.copy()
X_test = pca_xtest.copy()
y_train = y[:27000]
y_test = y[27000:]
X_train_new = (X_train - X_train.mean()) / X_train.std()
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
pipelines = make_pipeline(StandardScaler(), RandomForestRegressor(random_state=123))
rf_hyperparameters = { 
    'randomforestregressor__n_estimators' : [700, 1100],
    'randomforestregressor__max_features': ['auto', 'sqrt', 0.33],
}
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(pipelines, rf_hyperparameters, cv=10, n_jobs=-1)
model.fit(X_train, y_train)

# pickle.dump(model, open('model.pkl','wb'))
# pickle.dump(pca, open('pca.pkl','wb'))


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=123))]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_features': ['auto', 'sqrt',
                                                                 0.33],
                         'randomforestregressor__n_estimators': [100, 200]})

In [2]:
print('train accuracy',model.score(X_train,y_train))
print('test accuracy',model.score(X_test,y_test))

train accuracy 0.9978490491755017
test accuracy 0.9837099239342932
